## DSWD DROMIC Coronavirus disease (COVID-2019) situation reports

Reports from https://dromic.dswd.gov.ph/coronavirus-disease-covid-19-31-dec-2019/ in Tabular format.


In [6]:
import pandas as pd
import requests
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [7]:
# papermill parameters
output_folder = '../output/'

In [8]:
# Source Google Map DSWD Offices
dswd_offices = [
    ['DSWD Central Office', 'Central Office', 14.696294999999996, 121.09674800000005],
    ['DSWD National Resource Operation Center', 'NRLMB - NROC', 14.513659, 121.0036236],
    ['DSWD Visayas Disaster Response Center', 'NRLMB - VDRC', 10.3093678, 123.9049462],
    ['FO NCR Manila', 'NCR', 14.598674999999998, 120.990366],   
    ['FO 2 Tuguegarao, Cagayan', 'II', 17.655575000000006, 121.74608899999998],
    ['FO 1 San Fernando, La Union', 'I', 16.610573000000013, 120.317046],
    ['FO 3 San Fernando, Pampanga', 'III', 15.029500000000027, 120.68910000000005],
    ['FO 4A CALABARZON', 'CALABARZON', 14.425525999999985, 121.03137700000002],
    ['FO 4B MIMAROPA', 'MIMAROPA', 14.576845000000002, 120.99156300000004],
    ['FO 5 Legaspi City', 'V', 13.13603, 123.73976500000003],
    ['FO 6 Iloilo City', 'VI', 10.698014999999998, 122.54788800000006],
    ['FO 7 Cebu City', 'VII', 10.308428000000026, 123.90742999999998],
    ['FO 8 Tacloban City', 'VIII', 11.250660000000002, 125.00533799999994],
    ['FO 9 Zamboanga City', 'IX', 6.903621000000008, 122.08027400000003],
    ['FO 10 Cagayan de Oro', 'X', 8.44645800000001, 124.62133199999994],
    ['FO 11 Davao City', 'XI', 7.073695000000023, 125.61824999999999],
    ['FO 12 Koronadal', 'XII', 6.4894340000000295, 124.85539199999994],
    ['FO CARAGA, Butuan City', 'CARAGA', 8.94300700000001, 125.53416300000004],
    ['FO CAR Baguio City', 'CAR', 16.408525000000004, 120.60078599999997],
]
dswd_offices_df = pd.DataFrame(dswd_offices, columns=['name', 'alias', 'lat', 'lon'])

In [22]:
import os
if os.getcwd().endswith('notebooks'): root_path = '../'
else: root_path = ''
    
def path_of(path): 
    return root_path + path

In [23]:
from decimal import Decimal
from docx import Document
document = Document(path_of('datasets/sitreps/DSWD-DROMIC-Report-1-on-the-Coronavirus-Disease-COVID-19-as-of-20-March-2020-8PM-2.docx'))
tables = document.tables

# Available Standby Funds and Stockpiles
date0 = '2020-03-20-18'
sitrep_num0 = 1
table0 = tables[0]
data_start_row0 = 3
data0 = []
for i, row in enumerate(table0.rows):
    if i >= data_start_row0:
        cur_row = []
        cur_row.append(sitrep_num0)
        cur_row.append(datetime.strptime(date0, '%Y-%m-%d-%H'))
        for j, cell in enumerate(row.cells):
            p = cell.paragraphs[0] # get the first paragraph of the cell only
            if j == 0: # Office
                cur_row.append(p.text)
            else: # Numerical data
                if p.text.strip() == '-':
                    cur_row.append(None)
                else:
                    cur_row.append(Decimal(p.text.strip().replace(',', ''))) # remove commas and double
        data0.append(cur_row)

data0_df = pd.DataFrame(data0, columns=['sitrep', 'timestamp', 'office', 'standby_funds', 'ffp_quantity', 'ffp_total_cost', 'ofi_total_cost', 'nfi_total_cost', 'ofi_nfi_sub_total', 'total_funds_and_stockpile'])
data0_df = pd.merge(data0_df, dswd_offices_df, how='left', left_on='office', right_on='alias')
data0_df = data0_df.drop(['alias', 'name'], axis=1)
data0_df


,sitrep,timestamp,office,standby_funds,ffp_quantity,ffp_total_cost,ofi_total_cost,nfi_total_cost,ofi_nfi_sub_total,total_funds_and_stockpile,lat,lon
0,1,2020-03-20 18:00:00,Central Office,1253367735.97,None,None,None,None,None,1253367735.97,14.696295,121.096748
1,1,2020-03-20 18:00:00,NRLMB - NROC,None,84898,30734440.00,25033896.34,196090074.48,221123970.82,251858410.82,14.513659,121.003624
2,1,2020-03-20 18:00:00,NRLMB - VDRC,None,2567,941120.00,6747536.80,6358727.50,13106264.30,14047384.30,10.309368,123.904946
3,1,2020-03-20 18:00:00,I,None,4007,1488318.08,245144.64,36791330.65,37036475.29,38524793.37,16.610573,120.317046
4,1,2020-03-20 18:00:00,II,2829067.84,16562,5894250.24,554800.00,6764868.95,7319668.95,16042987.03,17.655575,121.746089
5,1,2020-03-20 18:00:00,III,3000550.00,8822,3615137.90,1956002.70,4735242.90,6691245.60,13306933.50,15.029500,120.689100
6,1,2020-03-20 18:00:00,CALABARZON,3000000.00,12050,3929189.60,1213850.00,8639451.49,9853301.49,16782491.09,14.425526,121.031377
7,1,2020-03-20 18:00:00,MIMAROPA,1451652.18,30615,13776750.00,5339530.60,6886713.04,12226243.64,27454645.82,14.576845,120.991563
8,1,2020-03-20 18:00:00,V,3000000.00,18538,5716656.94,5550788.00,31924347.16,37475135.16,46191792.10,13.136030,123.739765
9,1,2020-03-20 18:00:00,VI,1162699.50,18356,6521040.00,39145071.10,6394504.00,45539575.10,53223314.60,10.698015,122.547888


In [25]:
data0_df.to_csv('%s/dromic_covid19_sitreps_sitrep_%s_standby_funds_and_stockpiles_by_office.csv' % (output_folder, sitrep_num0), index = False)